OpenCV(정식 명칭 : Open Source Computer Vision Library)은 오픈 소스 컴퓨터 비전 라이브러리
1. 필터링
2. 행렬 계산
3. 객체 추적(Object Tracking)
4. 세분화(Segmentation)
5. 카메라 캘리브레이션(Calibration)
6. 특징점 추출
7. 물체 인식(Object recognition)
8. 기계 학습(Machine learning)
9. 파노라마(Stitching)
10. 컴퓨터 테이 셔널 포토 그래피(Computational Photography)

In [4]:
>>> import cv2 as cv
>>> print(cv.__version__)

4.5.1


In [2]:
#1. 이미지 파일을 읽고, 저장 imread(), imwrite()
from PIL import Image
import numpy as np
import cv2 as cv

im = cv.imread('c:\\opencv_test\\Lenna.png')
print(type(im))

print(im.dtype)
print(im.shape)  #행(높이), 열(폭), 색상(채널)
#색상의 순서 BGR
#im[:,:,(0,1)] = 0   0번째(B:파랑)과 1번쨰(G:초록)을 0(검정)으로 지정

im[:,:,(0,1)] = 0

#cv.imshow("Display window",im)

cv.imwrite("c:\\opencv_test\\cv2\\Lenna_b.png",im)

cv.imwrite("c:\\opencv_test\\cv2\\Lenna_b02.png",im,[cv.IMWRITE_JPEG_QUALITY,50])

cv.imwrite("c:\\opencv_test\\cv2\\Lenna_b03.png",im,[cv.IMWRITE_JPEG_QUALITY,90])

<class 'numpy.ndarray'>
uint8
(256, 256, 3)


True

In [9]:
#그레이 스케일(흑백처리) cv.IMREAD_GRAYSCALE / cv.cvtColor()
im = cv.imread('c:\\opencv_test\\Lenna.png')
print(type(im))

print(im.dtype)
print(im.shape)

im_gray = 0.299*im[:,:,2] + 0.587*im[:,:,1] + 0.114*im[:,:,0]
print(im_gray.shape)
print(im_gray.dtype)

cv.imwrite("c:\\opencv_test\\cv2\\Lenna_Y.png",im_gray)


<class 'numpy.ndarray'>
uint8
(256, 256, 3)
(256, 256)
float64


True

In [16]:
#imread()
im_gray_read = cv.imread('c:\\opencv_test\\1.jpg',cv.IMREAD_GRAYSCALE) #OS에 코덱에 따라 화소가 다르
print(type(im_gray_read))
print(im_gray_read.shape)

im = cv.imread('c:\\opencv_test\\1.jpg')
im_gray = cv.cvtColor(im,cv.COLOR_BGR2GRAY)  #BT.601 공식을 적용
print(type(im_gray))
print(im_gray.shape)

#판별
im_diff = im_gray.astype(int) - im_gray_read.astype(int)
print(im_diff.max())
print(im_diff.min())

<class 'numpy.ndarray'>
(683, 1024)
<class 'numpy.ndarray'>
(683, 1024)
10
-8


In [ ]:
#4. 이미지 결합 세로연결, 가로연결, 세로가로 바둑판 연결

#cv.vconcat()
#1. 폭이 동일한 이미지를 세로로 연결
#2. 같은 이미지를 반복세로 연결
#3. 폭이 서로 다른 이미지를 세로로 연결

# cv.hconcat()
#1. 높이가 동일한 이미지를 가로로 연결
#2. 같은 이미지를 반복 가로로 연결
#3. 높이가 서로 다른 이미지를 가로로 연결

# np.tile() : 같은 이미지를 반복정렬

In [23]:
#cv.vconcat()
#1. 같은 이미지를 세로로 연결
import cv2 as cv
import numpy as np

im1 = cv.imread('c:\\opencv_test\\Lenna.png')
im2 = cv.imread('c:\\opencv_test\\1.jpg')

im_v = cv.vconcat([im1,im1])
cv.imwrite("c:\\opencv_test\\cv2\\v_img.jpg",im_v)

im_v02 = np.tile(im1,(2,1,1))
cv.imwrite("c:\\opencv_test\\cv2\\v02_img.jpg",im_v02)

True

In [26]:

import cv2 as cv
import numpy as np

im1 = cv.imread('c:\\opencv_test\\Lenna.png')
im2 = cv.imread('c:\\opencv_test\\1.jpg')

#함수선언 resize(src, dsize[, dst[,fx[,fy[ interpolation]]]])
def my_resize(im_list,interpolation = cv.INTER_CUBIC):
    #1. 폭이 가장 작은 값을 리턴 받자.
    w_min = min(im.shape[1] for im in im_list)
    
    #2. 폭의 사이즈를 재조정
    im_list_resize = [cv.resize(im,(w_min,int(im.shape[0]*w_min/im.shape[1])),
                                interpolation=interpolation) for im in im_list]
    
    return cv.vconcat(im_list_resize) #결합해서 리턴

im_v = my_resize([im1,im2,im1])

cv.imwrite("c:\\opencv_test\\cv2\\v_img_resize.jpg",im_v)


True

In [29]:

import cv2 as cv
import numpy as np

im1 = cv.imread('c:\\opencv_test\\Lenna.png')
im2 = cv.imread('c:\\opencv_test\\1.jpg')

#함수선언 resize(src, dsize[, dst[,fx[,fy[ interpolation]]]])
def my_hresize(im_list,interpolation = cv.INTER_CUBIC):
    #1. 높이가 가장 작은 값을 리턴 받자.
    h_min = min(im.shape[0] for im in im_list)
    
    #2. 폭의 사이즈를 재조정
    im_list_resize = [cv.resize(im,(int(im.shape[1]*h_min/im.shape[0]),h_min),
                                interpolation=interpolation) for im in im_list]
    
    return cv.hconcat(im_list_resize) #결합해서 리턴

im_h = my_hresize([im1,im2,im1])

cv.imwrite("c:\\opencv_test\\cv2\\h_img_resize.jpg",im_h)


True

In [32]:
#바둑판식 타일형을 만들어 보자.
import cv2 as cv
import numpy as np

image = cv.imread('c:\\opencv_test\\Lenna.png')

def my_tile(im_list_2d):
    return cv.vconcat([cv.hconcat(im_list_h) for im_list_h in im_list_2d])


im1 = cv.resize(image,dsize=(0,0),fx=0.5,fy=0.5) #fx = 수평측 비율            fy = 수직측 비율

im_tile = my_tile([[im1,im1,im1,im1],
                   [im1,im1,im1,im1],
                   [im1,im1,im1,im1]])

cv.imwrite("c:\\opencv_test\\cv2\\tile_img.jpg",im_tile)


True

In [37]:
#이미지회전 cv.rotate(ndarray,rotatecode), np.rot90
#상하 좌우 반전 cv.flip(ndarray,flipcode), np.flip()
#flipcode = 0 : 상하반전
#flipcode > 0 : 좌우반전
#flipcode < 0 : 상하좌우반전

image = cv.imread('c:\\opencv_test\\Lenna.png')
print(image.shape)

#img_r_90 = cv.rotate(image, cv.ROTATE_90_CLOCKWISE)
#cv.imwrite("c:\\opencv_test\\cv2\\Lenna_90_img.jpg",img_r_90)

img_flip_ud = cv.flip(image, 0)
cv.imwrite("c:\\opencv_test\\cv2\\Lenna_ud_img.jpg",img_flip_ud)

img_flip_lr = cv.flip(image, 1)
cv.imwrite("c:\\opencv_test\\cv2\\Lenna_lr_img.jpg",img_flip_lr)

img_flip_udlr = cv.flip(image, -1)
cv.imwrite("c:\\opencv_test\\cv2\\Lenna_udlr_img.jpg",img_flip_udlr)

(256, 256, 3)


True